In [1]:
import tkinter as tk
from tkinter import messagebox
import random
import time
import copy

# --------------------------- GAME LOGIC ---------------------------

class QuanGame:
    def __init__(self):
        # Initialize game state
        self.o = [0] * 13  # 1-12
        self.q = [False] * 13
        self.q[6] = True
        self.q[12] = True
        self.quan = [None] * 13  # Labels for special pits
        self.gold_me = 0
        self.gold_bot = 0
        self.time_bh = 0
        self.time_limit = 0
        self.root = None
        self.gui_elements = {}
        self.oke = False  # Flag to indicate if the game has started
        self.current_turn = 'player'  # 'player' or 'bot'

    # --------------------------- GUI FUNCTIONS ---------------------------

    def o_r(self, vt):
        """Read the value of pit vt."""
        return self.o[vt]

    def o_w(self, vt, gt):
        """Write the value gt to pit vt."""
        self.o[vt] = gt
        self.gui_elements['button'][vt].config(text='' if gt == 0 else str(gt))

    def showw(self, me, w):
        """Update the status label."""
        self.gui_elements['show_tt'].config(text=f"{me}: {w}")

    def luot(self, type):
        """Update turn indicators."""
        if type == 1:  # Player's turn
            self.gui_elements['lb_var'].config(state='normal')
            self.gui_elements['lt_var'].config(state='disabled')
            self.gui_elements['lb_var'].var.set(1)
            self.gui_elements['lt_var'].var.set(0)
        else:  # BOT's turn
            self.gui_elements['lb_var'].config(state='disabled')
            self.gui_elements['lt_var'].config(state='normal')
            self.gui_elements['lb_var'].var.set(0)
            self.gui_elements['lt_var'].var.set(1)

    def get_level(self):
        """Get the selected difficulty level."""
        val = self.gui_elements['difficulty'].get()
        if val == 1:
            return 30
        elif val == 2:
            return 25
        elif val == 3:
            return 15
        elif val == 4:
            return 5
        else:
            return 25  # Default value

    def get_sec(self):
        """Get current time in seconds since epoch."""
        return int(time.time())

    def tinh_thoi_gian(self):
        """Calculate remaining time."""
        conlai = self.time_limit - (self.get_sec() - self.time_bh)
        self.gui_elements['stime'].config(text=f"Thời gian còn lại: {conlai}")
        if conlai <= 0:
            return False
        return True

    def chon_huong(self, o_so):
        """Choose direction for the move."""
        self.showw('', '')
        direction = None

        def select_direction(dir):
            nonlocal direction
            direction = dir
            self.gui_huong.destroy()

        self.gui_huong = tk.Toplevel(self.root)
        self.gui_huong.title(f"Bạn chọn ô số {o_so}!")
        tk.Label(self.gui_huong, text=f"Hãy chọn hướng đi cho ô số {o_so}").pack(pady=10)
        tk.Button(self.gui_huong, text="<<<", command=lambda: select_direction(-1)).pack(side=tk.LEFT, padx=20, pady=20)
        tk.Button(self.gui_huong, text=">>>", command=lambda: select_direction(1)).pack(side=tk.RIGHT, padx=20, pady=20)
        self.root.wait_window(self.gui_huong)
        return direction

    def xoay_vong(self, vt, huongdi, so_o=12):
        """Rotate around the pits."""
        vt += huongdi
        if vt <= 0:
            return so_o
        if vt > so_o:
            return 1
        return vt

    def process(self, o_so):
        """Process player's move."""
        if self.o_r(o_so) > 0:
            self.current_turn = 'player'
            huongdi = self.chon_huong(o_so)
            if huongdi is None:
                return  # Player closed the direction selection window
            print(f"Player selected pit {o_so} with direction {huongdi}")
            earned = self.buoc_di('Bạn', o_so, huongdi)
            self.gold_me += earned
            self.check_o()
            self.gui_elements['s_gm'].config(text=f"bạn: {self.gold_me} !")
            self.time_bh = self.get_sec()
            # After player's move, trigger BOT's move
            self.current_turn = 'bot'
            self.luot(0)
            self.root.after(1000, self.ai_move_minimax)  # BOT makes a move after 1 second
        else:
            messagebox.showerror("Lỗi", "Bạn không thể chọn ô không có tiền")

    def buoc_di(self, me, o_so, huongdi):
        """Execute a move from pit o_so in direction huongdi."""
        giatri = self.o_r(o_so)
        hientai = o_so
        self.o_w(hientai, 0)
        an = 0
        while True:
            hientai = self.xoay_vong(hientai, huongdi)
            self.showw(me, f"Đang ở ô {hientai}, số quân đang dải {giatri}")
            current = self.o_r(hientai)
            self.o_w(hientai, current + 1)
            giatri -= 1
            if giatri <= 0:
                hientai = self.xoay_vong(hientai, huongdi)
                if self.o_r(hientai) == 0:
                    while self.o_r(hientai) == 0:
                        hientai = self.xoay_vong(hientai, huongdi)
                        if self.o_r(hientai) == 0:
                            break
                        an += self.o_r(hientai)
                        self.showw(me, f'ăn được {self.o_r(hientai)} quân tại ô {hientai}')
                        self.o_w(hientai, 0)
                        hientai = self.xoay_vong(hientai, huongdi)
                        self.root.update()
                        time.sleep(0.5)
                    break
                else:
                    if hientai in [6, 12]:
                        if self.q[hientai] and (self.o_r(hientai) - 10 <= 0):
                            break
                        elif self.o_r(hientai) <= 1:
                            break
                        else:
                            giatri = 1
                            self.o_w(hientai, self.o_r(hientai) - 1)
                    else:
                        giatri = self.o_r(hientai)
                        self.o_w(hientai, 0)
            self.root.update()
            time.sleep(0.5)
        if an > 0:
            self.showw(me, f'Ăn được {an} quân!')
        else:
            self.showw(me, 'Chẫng!')
        return an

    def check_o(self):
        """Check the game state after each move."""
        all_empty = all(self.o_r(i) == 0 for i in range(1, 13))
        if all_empty:
            if self.gold_me > self.gold_bot:
                messagebox.showinfo("Thông báo", "Xin chúc mừng bạn là người thắng cuộc :))!")
            elif self.gold_me == self.gold_bot:
                messagebox.showinfo("Thông báo", "Haizz, trận này hoà cmnr hihi :))!")
            else:
                messagebox.showinfo("Thông báo", "Tèn tén ten, bạn thua rồi haha :))!")
            self.root.destroy()
            exit()
        # Check if player pits are empty
        if all(self.o_r(i) == 0 for i in range(1, 6)):
            if self.gold_me >= 5:
                self.showw('Bạn', 'đã hết quân, đã dải mỗi ô 1 quân!')
                for i in range(1, 6):
                    self.o_w(i, 1)
                    self.gold_me -= 1
                    self.gui_elements['s_gm'].config(text=f"bạn: {self.gold_me} !")
                    self.root.update()
                    time.sleep(0.5)
            else:
                messagebox.showinfo("Thông báo", "Bạn không đủ tiền để dải quân, BẠN THUA :))")
                self.root.destroy()
                exit()
        # Check if BOT pits are empty
        if all(self.o_r(i) == 0 for i in range(7, 12)):
            if self.gold_bot >= 5:
                self.showw('BOT', 'đã hết quân, đã dải mỗi ô 1 quân!')
                for i in range(7, 12):
                    self.o_w(i, 1)
                    self.gold_bot -= 1
                    self.gui_elements['s_gb'].config(text=f"tôi: {self.gold_bot} !")
                    self.root.update()
                    time.sleep(0.5)
            else:
                messagebox.showinfo("Thông báo", "Tớ không đủ tiền để dải quân, TỚ THUA RỒI HUHU :((")
                self.root.destroy()
                exit()

    # --------------------------- MINIMAX ALGORITHM ---------------------------

    def minimax(self, state, depth, is_maximizing):
        """
        Minimax algorithm with depth limitation.
        :param state: Current game state (copy of self.o).
        :param depth: Current depth in the game tree.
        :param is_maximizing: Boolean indicating if the current layer is maximizing or minimizing.
        :return: The evaluation score.
        """
        # Terminal condition: depth == 0 or game over
        if depth == 0 or self.is_game_over(state):
            return self.evaluate(state)

        if is_maximizing:
            max_eval = -float('inf')
            for move in range(7, 12):
                if state[move] > 0:
                    new_state, earned = self.simulate_move(state, move, 'bot')
                    eval = self.minimax(new_state, depth - 1, False) + earned
                    max_eval = max(max_eval, eval)
            return max_eval
        else:
            min_eval = float('inf')
            for move in range(1, 6):
                if state[move] > 0:
                    new_state, earned = self.simulate_move(state, move, 'player')
                    eval = self.minimax(new_state, depth - 1, True) - earned
                    min_eval = min(min_eval, eval)
            return min_eval

    def is_game_over(self, state):
        """Check if the game is over."""
        return all(state[i] == 0 for i in range(1, 13))

    def evaluate(self, state):
        """Evaluation function: difference in gold."""
        return self.gold_bot - self.gold_me

    def simulate_move(self, state, move, player):
        """
        Simulate a move and return new state and earned coins.
        :param state: Current game state.
        :param move: Pit number to move from.
        :param player: 'player' or 'bot'.
        :return: (new_state, earned_coins)
        """
        new_state = copy.deepcopy(state)
        coins = new_state[move]
        new_state[move] = 0
        hientai = move
        huongdi = -1 if player == 'bot' else 1  # BOT đi ngược hướng

        earned = 0
        while coins > 0:
            hientai = self.xoay_vong(hientai, huongdi)
            new_state[hientai] += 1
            coins -= 1

        if coins == 0:
            hientai = self.xoay_vong(hientai, huongdi)
            if new_state[hientai] == 1:
                if hientai in [6, 12]:
                    if player == 'bot':
                        earned += new_state[hientai]
                        new_state[hientai] = 0
            elif hientai not in [6, 12]:
                earned += new_state[hientai]
                new_state[hientai] = 0

        return new_state, earned

    def ai_move_minimax(self):
        """AI makes a move using the Minimax algorithm."""
        best_score = -float('inf')
        best_move = None
        for move in range(7, 12):
            if self.o_r(move) > 0:
                new_state, earned = self.simulate_move(self.o, move, 'bot')
                score = self.minimax(new_state, 2, False) + earned
                print(f"AI evaluating move {move}: score {score}")
                if score > best_score:
                    best_score = score
                    best_move = move

        if best_move:
            print(f"AI selected move {best_move} with score {best_score}")
            earned = self.buoc_di('BOT', best_move, -1)
            self.gold_bot += earned
            self.gui_elements['s_gb'].config(text=f"tôi: {self.gold_bot} !")
            self.check_o()
            self.time_bh = self.get_sec()
            self.current_turn = 'player'
            self.luot(1)
        else:
            messagebox.showinfo("Thông báo", "BOT không có nước đi nào hợp lệ!")

    # --------------------------- RANDOM DISTRIBUTION ---------------------------

    def rand_quan(self):
        """Randomly distribute the coins if selected."""
        ox = [True] * 13
        for thien in range(1, 3):
            if thien == 1:
                st, en = 1, 5
            else:
                st, en = 7, 11
            tong = 0
            for i in range(st, en + 1):
                if i != en:
                    while True:
                        ngaunhien = random.randint(0, 7)
                        if ngaunhien >= 3 and (tong + ngaunhien) <= 25:
                            break
                    while True:
                        ochon = random.randint(st, en)
                        if ox[ochon]:
                            break
                    self.o_w(ochon, ngaunhien if ngaunhien != 0 else 0)
                    tong += ngaunhien
                    ox[ochon] = False
                else:
                    while True:
                        ochon = random.randint(st, en)
                        if ox[ochon]:
                            break
                    self.o_w(ochon, 25 - tong)

    # --------------------------- GAME FUNCTIONS ---------------------------

    def main(self):
        """Initialize and run the main game window."""
        self.root = tk.Tk()
        self.root.title("Ô Ăn Quan QT")

        # Create buttons for pits
        self.gui_elements['button'] = {}
        positions = {
            10: (216, 56),
            8: (312, 56),
            9: (264, 56),
            7: (360, 56),
            4: (312, 104),
            3: (264, 104),
            11: (168, 56),
            1: (168, 104),
            5: (360, 104),
            2: (216, 104),
            6: (416, 56),
            12: (120, 56)
        }
        for key, (x, y) in positions.items():
            initial = "0" if key in [6, 12] else "5"
            self.o[key] = 0 if key in [6, 12] else 5
            btn = tk.Button(self.root, text=initial, width=5, height=2,
                            command=lambda k=key: self.on_button_click(k))
            if key in [6, 12]:  # Adjust size for special pits
                btn.config(width=4, height=5)  # Example sizes, adjust as needed
            btn.place(x=x, y=y)
            self.gui_elements['button'][key] = btn

        # Labels for pits numbers
        for i in range(1, 6):
            tk.Label(self.root, text=str(i)).place(x=184 + 48*(i-1), y=144)

        # Labels for special pits (6 and 12)
        self.quan[6] = tk.Label(self.root, text="(Q)" if self.q[6] else "")
        self.quan[6].place(x=464, y=88)
        self.quan[12] = tk.Label(self.root, text="(Q)" if self.q[12] else "")
        self.quan[12].place(x=96, y=88)

        # Radio buttons for difficulty
        self.gui_elements['difficulty'] = tk.IntVar(value=1)  # Default to "Dễ"

        tk.Radiobutton(self.root, text="Dễ", variable=self.gui_elements['difficulty'], value=1).place(x=504, y=40)
        tk.Radiobutton(self.root, text="Trung bình", variable=self.gui_elements['difficulty'], value=2).place(x=504, y=64)
        tk.Radiobutton(self.root, text="Khó", variable=self.gui_elements['difficulty'], value=3).place(x=504, y=88)
        tk.Radiobutton(self.root, text="Ác mộng", variable=self.gui_elements['difficulty'], value=4).place(x=504, y=112)

        # Checkbox for random
        self.gui_elements['rand'] = tk.IntVar()
        tk.Checkbutton(self.root, text="Random", variable=self.gui_elements['rand']).place(x=504, y=136)

        # Start button
        self.gui_elements['start'] = tk.Button(self.root, text="Chơi", command=self.start_game)
        self.gui_elements['start'].place(x=504, y=176)

        # Labels for gold
        self.gui_elements['s_gb'] = tk.Label(self.root, text=f"tôi: {self.gold_bot} !")
        self.gui_elements['s_gb'].place(x=8, y=72)
        self.gui_elements['s_gm'] = tk.Label(self.root, text=f"bạn: {self.gold_me} !")
        self.gui_elements['s_gm'].place(x=8, y=136)

        # Turn indicators
        self.gui_elements['lb_var'] = tk.Checkbutton(self.root, text="Lượt của bạn", state='disabled')
        self.gui_elements['lb_var'].var = tk.IntVar()
        self.gui_elements['lb_var'].config(variable=self.gui_elements['lb_var'].var)
        self.gui_elements['lb_var'].place(x=88, y=184)

        self.gui_elements['lt_var'] = tk.Checkbutton(self.root, text="Lượt của BOT", state='disabled')
        self.gui_elements['lt_var'].var = tk.IntVar()
        self.gui_elements['lt_var'].config(variable=self.gui_elements['lt_var'].var)
        self.gui_elements['lt_var'].place(x=88, y=8)

        # Time remaining
        self.gui_elements['stime'] = tk.Label(self.root, text="")
        self.gui_elements['stime'].place(x=224, y=184)

        # Show status
        self.gui_elements['show_tt'] = tk.Label(self.root, text="")
        self.gui_elements['show_tt'].place(x=256, y=8)

        self.root.geometry("586x232+269+232")
        self.root.resizable(False, False)
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        self.root.mainloop()

    def start_game(self):
        """Start the game when the 'Chơi' button is clicked."""
        messagebox.showinfo("Thông báo", "Tôi nhường bạn chơi trước, hãy chú ý thời gian nhé!")
        self.oke = True
        self.time_limit = self.get_level()
        self.time_bh = self.get_sec()
        if self.gui_elements['rand'].get() == 1:
            self.rand_quan()
        # Delete the start button
        self.gui_elements['start'].destroy()
        # Indicate it's player's turn
        self.luot(1)

    def on_close(self):
        """Handle window close event."""
        self.root.destroy()
        exit()

    def on_button_click(self, key):
        """Handle pit button clicks."""
        if self.oke and 1 <= key <= 5 and self.current_turn == 'player':
            self.process(key)

    # --------------------------- MINIMAX ALGORITHM ---------------------------

    def minimax(self, state, depth, is_maximizing):
        """
        Minimax algorithm with depth limitation.
        :param state: Current game state (copy of self.o).
        :param depth: Current depth in the game tree.
        :param is_maximizing: Boolean indicating if the current layer is maximizing or minimizing.
        :return: The evaluation score.
        """
        # Terminal condition: depth == 0 or game over
        if depth == 0 or self.is_game_over(state):
            return self.evaluate(state)

        if is_maximizing:
            max_eval = -float('inf')
            for move in range(7, 12):
                if state[move] > 0:
                    new_state, earned = self.simulate_move(state, move, 'bot')
                    eval = self.minimax(new_state, depth - 1, False) + earned
                    max_eval = max(max_eval, eval)
            return max_eval
        else:
            min_eval = float('inf')
            for move in range(1, 6):
                if state[move] > 0:
                    new_state, earned = self.simulate_move(state, move, 'player')
                    eval = self.minimax(new_state, depth - 1, True) - earned
                    min_eval = min(min_eval, eval)
            return min_eval

    def is_game_over(self, state):
        """Check if the game is over."""
        return all(state[i] == 0 for i in range(1, 13))

    def evaluate(self, state):
        """Evaluation function: difference in gold."""
        return self.gold_bot - self.gold_me

    def simulate_move(self, state, move, player):
        """
        Simulate a move and return new state and earned coins.
        :param state: Current game state.
        :param move: Pit number to move from.
        :param player: 'player' or 'bot'.
        :return: (new_state, earned_coins)
        """
        new_state = copy.deepcopy(state)
        coins = new_state[move]
        new_state[move] = 0
        hientai = move
        huongdi = -1 if player == 'bot' else 1  # BOT đi ngược hướng

        earned = 0
        while coins > 0:
            hientai = self.xoay_vong(hientai, huongdi)
            new_state[hientai] += 1
            coins -= 1

        if coins == 0:
            hientai = self.xoay_vong(hientai, huongdi)
            if new_state[hientai] == 1:
                if hientai in [6, 12]:
                    if player == 'bot':
                        earned += new_state[hientai]
                        new_state[hientai] = 0
            elif hientai not in [6, 12]:
                earned += new_state[hientai]
                new_state[hientai] = 0

        return new_state, earned

    def ai_move_minimax(self):
        """AI makes a move using the Minimax algorithm."""
        best_score = -float('inf')
        best_move = None
        for move in range(7, 12):
            if self.o_r(move) > 0:
                new_state, earned = self.simulate_move(self.o, move, 'bot')
                score = self.minimax(new_state, 2, False) + earned
                print(f"AI evaluating move {move}: score {score}")
                if score > best_score:
                    best_score = score
                    best_move = move

        if best_move:
            print(f"AI selected move {best_move} with score {best_score}")
            earned = self.buoc_di('BOT', best_move, -1)
            self.gold_bot += earned
            self.gui_elements['s_gb'].config(text=f"tôi: {self.gold_bot} !")
            self.check_o()
            self.time_bh = self.get_sec()
            self.current_turn = 'player'
            self.luot(1)
        else:
            messagebox.showinfo("Thông báo", "BOT không có nước đi nào hợp lệ!")

    # --------------------------- RANDOM DISTRIBUTION ---------------------------

    def rand_quan(self):
        """Randomly distribute the coins if selected."""
        ox = [True] * 13
        for thien in range(1, 3):
            if thien == 1:
                st, en = 1, 5
            else:
                st, en = 7, 11
            tong = 0
            for i in range(st, en + 1):
                if i != en:
                    while True:
                        ngaunhien = random.randint(0, 7)
                        if ngaunhien >= 3 and (tong + ngaunhien) <= 25:
                            break
                    while True:
                        ochon = random.randint(st, en)
                        if ox[ochon]:
                            break
                    self.o_w(ochon, ngaunhien if ngaunhien != 0 else 0)
                    tong += ngaunhien
                    ox[ochon] = False
                else:
                    while True:
                        ochon = random.randint(st, en)
                        if ox[ochon]:
                            break
                    self.o_w(ochon, 25 - tong)

# --------------------------- RUN GAME ---------------------------

if __name__ == "__main__":
    game = QuanGame()
    game.main()


Player selected pit 3 with direction -1
AI evaluating move 7: score -11
AI evaluating move 8: score -1
AI evaluating move 10: score -5
AI evaluating move 11: score 1
AI selected move 11 with score 1
